# Fantasy Premier League team optimisation
### Intro

This is an attempt to optimise my fantasy football squad selection, picking an initial team and subsequent transfers. I've used data from https://github.com/vaastav/Fantasy-Premier-League/ which includes an 'xP' field, which isn't exactly what's required but is a good enough placeholder until the season starts with more accurate player predictions.

Player points prediction is probably the most technical element of a task like this. It would involve not only gathering the most common data but also searching for alternative data that could improve accuracy, such as betting odds for various markets or even something as stupid as the tweets of the player may improve accuracy. There are a number of available models currently out there, put together by people who have thought a lot more about the problem than I have, so initially this project is purely an optimisation problem based on the predicted points and any other factors useful for creating a team.

The optimisation is carried out using `pulp`. I'd considered converting the problem into `pytorch` to crunch the numbers on a GPU, but it's only taking 5 seconds or so to run over a CPU for 6 gameweeks which is fast enough for now.

First grabbing the data:
```
git clone https://github.com/vaastav/Fantasy-Premier-League.git
```

### Data prep

In [3]:
import pulp
import pandas as pd
import chardet
import os
import seaborn as sns
import matplotlib.pyplot as plt
import requests

Load data from fpl api

In [14]:
r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
fpl_data = r.json()
element_data = pd.DataFrame(fpl_data['elements'])

In [16]:
element_data.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,NaN,None,438098,0,0,0,0,0,3,3.0,...,140,86,465,187,250,104,456,152,0.62,0.31
1,NaN,None,205651,0,0,0,0,0,4,3.2,...,29,12,544,54,143,22,177,31,1.04,0.67
2,NaN,None,226597,0,0,0,0,0,2,4.2,...,81,6,499,186,42,6,27,10,1.01,0.47
3,NaN,None,219847,0,0,0,0,0,4,3.5,...,12,4,563,65,18,4,24,5,1.03,0.55
4,NaN,None,463748,0,0,0,0,0,1,2.0,...,554,60,55,23,445,52,370,56,0.00,0.00


In [17]:
type_data = pd.DataFrame(fpl_data['element_types']).set_index(['id'])
type_data.head()

,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
id,,,,,,,,,,,,
1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],63
2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],190
3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],253
4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],68


### Optimisation
#### Single gameweek optimisation

First we'll just build an optimisation model for a single gameweek

In [4]:
df = pd.read_csv('../data/fpl_predictions.csv')
df = df.melt(id_vars=["name", "team", "position", "value", "pts1-6", "value1-6"], 
                    value_vars=["1", "2", "3", "4", "5", "6"],
                    var_name="gameweek", 
                    value_name="xP")
df['season'] = "2024-25"
df = df.drop(columns=['pts1-6', 'value1-6'])

df.head()

,name,team,position,value,gameweek,xP,season
0,M.Salah,LIV,MID,12.5,1,7.0,2024-25
1,Haaland,MCI,FWD,15.0,1,5.5,2024-25
2,Palmer,CHE,MID,10.5,1,4.9,2024-25
3,Saka,ARS,MID,10.0,1,6.8,2024-25
4,Son,TOT,MID,10.0,1,5.7,2024-25


In [5]:
cols_opt = [
    "name",
    "position",
    "team",
    "xP",
    "value",
    "gameweek",
    "season"
    ]

df_optimisation = df[cols_opt].sort_values(by=['gameweek', 'season'], ascending=[True, True])
df_optimisation['gameweek'] = df_optimisation['gameweek'].astype('int')
df_optimisation['value'] = df_optimisation['value'].astype('float64')
df_optimisation['xP'] = df_optimisation['xP'].astype('float64')
print(df_optimisation)

          name position team   xP  value  gameweek   season
0      M.Salah      MID  LIV  7.0   12.5         1  2024-25
1      Haaland      FWD  MCI  5.5   15.0         1  2024-25
2       Palmer      MID  CHE  4.9   10.5         1  2024-25
3         Saka      MID  ARS  6.8   10.0         1  2024-25
4          Son      MID  TOT  5.7   10.0         1  2024-25
...        ...      ...  ...  ...    ...       ...      ...
3415    Downes      MID  SOT  0.0    5.0         6  2024-25
3416    Austin       GK  TOT  0.0    4.0         6  2024-25
3417  Reguilon      DEF  TOT  0.0    4.5         6  2024-25
3418  Whiteman       GK  TOT  0.0    4.0         6  2024-25
3419      King       GK  WOL  0.0    4.0         6  2024-25

[3420 rows x 7 columns]


In [8]:
def solve_single_period_fpl(budget, data, gameweek, season):
    # Filter data for the specified gameweek and season
    filtered_data = data[(data['gameweek'] == gameweek) & (data['season'] == season)]
    
    # Create the model
    model = pulp.LpProblem("FPL_Optimization", pulp.LpMaximize)

    # Variables
    squad = pulp.LpVariable.dicts("squad", filtered_data.index, cat='Binary')
    lineup = pulp.LpVariable.dicts("lineup", filtered_data.index, cat='Binary')
    captain = pulp.LpVariable.dicts("captain", filtered_data.index, cat='Binary')
    vicecap = pulp.LpVariable.dicts("vicecap", filtered_data.index, cat='Binary')

    # Objective
    model += pulp.lpSum(filtered_data.loc[p, 'xP'] * (lineup[p] + captain[p] + 0.1 * vicecap[p]) for p in filtered_data.index)

    # Constraints
    # Squad size
    model += pulp.lpSum(squad[p] for p in filtered_data.index) == 15

    # Lineup size
    model += pulp.lpSum(lineup[p] for p in filtered_data.index) == 11

    # Captain and vice-captain
    model += pulp.lpSum(captain[p] for p in filtered_data.index) == 1
    model += pulp.lpSum(vicecap[p] for p in filtered_data.index) == 1

    # Lineup, captain, and vice-captain must be in squad
    for p in filtered_data.index:
        model += lineup[p] <= squad[p]
        model += captain[p] <= lineup[p]
        model += vicecap[p] <= lineup[p]
        model += captain[p] + vicecap[p] <= 1

    # Position constraints
    for pos in filtered_data['position'].unique():
        model += pulp.lpSum(squad[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) == {'GK': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}[pos]
        if pos == 'GK':
            model += pulp.lpSum(lineup[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) == 1
        elif pos in ['DEF', 'MID']:
            model += pulp.lpSum(lineup[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) >= 3
            model += pulp.lpSum(lineup[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) <= 5
        else:  # FWD
            model += pulp.lpSum(lineup[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) >= 1
            model += pulp.lpSum(lineup[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) <= 3

    # Budget constraint (player prices are 10x)
    model += pulp.lpSum(filtered_data.loc[p, 'value'] * squad[p] for p in filtered_data.index) <= budget

    # Team limit
    for team in filtered_data['team'].unique():
        model += pulp.lpSum(squad[p] for p in filtered_data.index if filtered_data.loc[p, 'team'] == team) <= 3

    # Solve the model
    model.solve()

    # Process results
    if pulp.LpStatus[model.status] == 'Optimal':
        picks = []
        for p in filtered_data.index:
            if squad[p].value() > 0.5:
                player_data = filtered_data.loc[p]
                is_captain = 1 if captain[p].value() > 0.5 else 0
                is_lineup = 1 if lineup[p].value() > 0.5 else 0
                is_vice = 1 if vicecap[p].value() > 0.5 else 0
                picks.append([
                    player_data['name'],
                    player_data['position'],
                    player_data['team'],
                    player_data['value'],
                    player_data['xP'],
                    is_lineup,
                    is_captain,
                    is_vice
                ])

        picks_df = pd.DataFrame(picks, columns=['name', 'pos', 'team', 'price', 'xP', 'lineup', 'captain', 'vicecaptain'])
        picks_df = picks_df.sort_values(by=['lineup', 'pos', 'xP'], ascending=[False, True, False])
        
        total_xp = pulp.value(model.objective)
        print(f'Total expected points for budget {budget}: {total_xp}')

        return {'model': model, 'picks': picks_df, 'total_xp': total_xp}
    else:
        print(f"Optimization failed. Status: {pulp.LpStatus[model.status]}")
        return None


In [9]:
results = solve_single_period_fpl(100, df_optimisation, 1, '2024-25')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/c5d83d70cd064829a13dd93916916f28-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/c5d83d70cd064829a13dd93916916f28-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2321 COLUMNS
At line 17858 RHS
At line 20175 BOUNDS
At line 22456 ENDATA
Problem MODEL has 2316 rows, 2280 columns and 9629 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 66.205 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1710 strengthened rows, 0 substitutions
Cgl0004I processed model has 1177 rows, 2280 columns (2280 integer (2280 of which binary)) and 7422 elements
Cutoff increment increased from 1

In [10]:
if results is not None:
    picks = results['picks']
    print(picks)
else:
    print("No valid solution found.")
    
# Get total expected points of the squad
total_xp = round(picks['xP'].sum(),2)

# Get total cost of the squad
total_cost = picks['price'].sum() / 10

              name  pos team  price   xP  lineup  captain  vicecaptain
9         Trippier  DEF  NEW    6.0  5.2       1        0            0
7          Gabriel  DEF  ARS    6.0  4.6       1        0            0
8      Pedro Porro  DEF  TOT    5.5  4.2       1        0            0
5             Isak  FWD  NEW    8.5  5.9       1        0            0
6             Wood  FWD  NOT    6.0  4.7       1        0            0
10            Pope   GK  NEW    5.0  3.9       1        0            0
0          M.Salah  MID  LIV   12.5  7.0       1        1            0
1             Saka  MID  ARS   10.0  6.8       1        0            1
2         Ødegaard  MID  ARS    8.5  5.8       1        0            0
3      B.Fernandes  MID  MUN    8.5  5.7       1        0            0
4              Eze  MID  CRY    7.0  4.7       1        0            0
11  Harwood-Bellis  DEF  SOT    4.0  1.4       0        0            0
12      Woolfenden  DEF  IPS    4.0  1.4       0        0            0
13    

So above shows the optimised squad and starting lineup for gameweek 1 of the 2023-24 season, given a budget of 100. The total xP for the squad is 47.1 at a cost of 100.0

#### Multi-period optimisation
Now for the trickier task of optimising over multiple gameweeks, initially just assuming a wildcard (ie no current team) over n gameweeks.

In [18]:
def solve_multi_period_fpl(budget, data, start_gameweek, end_gameweek, season):
    # Filter data for the specified gameweeks and season
    filtered_data = data[(data['gameweek'] >= start_gameweek) & 
                         (data['gameweek'] <= end_gameweek) & 
                         (data['season'] == season)]
    
    # Create a dictionary for quick access to player data
    player_data = {(row['name'], row['gameweek']): row for _, row in filtered_data.iterrows()}
    
    # Create the model
    model = pulp.LpProblem("Multi_Period_FPL_Optimization", pulp.LpMaximize)

    # Sets
    gameweeks = range(start_gameweek, end_gameweek + 1)
    players = filtered_data['name'].unique()
    positions = filtered_data['position'].unique()
    teams = filtered_data['team'].unique()

    # Variables
    squad = pulp.LpVariable.dicts("squad", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    lineup = pulp.LpVariable.dicts("lineup", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    captain = pulp.LpVariable.dicts("captain", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    vicecap = pulp.LpVariable.dicts("vicecap", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    transfer_in = pulp.LpVariable.dicts("transfer_in", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    transfer_out = pulp.LpVariable.dicts("transfer_out", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    free_transfers = pulp.LpVariable.dicts("free_transfers", gameweeks, lowBound=0, upBound=5, cat='Integer')
    paid_transfers = pulp.LpVariable.dicts("paid_transfers", gameweeks, lowBound=0, cat='Integer')

    # Objective
    model += pulp.lpSum(player_data.get((p, gw), {}).get('xP', 0) * 
                        (lineup[p, gw] + captain[p, gw] + 0.1 * vicecap[p, gw]) 
                        for p in players for gw in gameweeks) - \
             pulp.lpSum(4 * paid_transfers[gw] for gw in gameweeks)
    
    model += free_transfers[start_gameweek] == 0

    # Constraints
    for gw in gameweeks:
        # Squad size
        model += pulp.lpSum(squad[p, gw] for p in players) == 15

        # Lineup size
        model += pulp.lpSum(lineup[p, gw] for p in players) == 11

        # Captain and vice-captain
        model += pulp.lpSum(captain[p, gw] for p in players) == 1
        model += pulp.lpSum(vicecap[p, gw] for p in players) == 1

        # Lineup, captain, and vice-captain must be in squad
        for p in players:
            model += lineup[p, gw] <= squad[p, gw]
            model += captain[p, gw] <= lineup[p, gw]
            model += vicecap[p, gw] <= lineup[p, gw]
            model += captain[p, gw] + vicecap[p, gw] <= 1

        # Position constraints
        for pos in positions:
            model += pulp.lpSum(squad[p, gw] for p in players if player_data.get((p, gw), {}).get('position') == pos) == {'GK': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}[pos]
            if pos == 'GK':
                model += pulp.lpSum(lineup[p, gw] for p in players if player_data.get((p, gw), {}).get('position') == pos) == 1
            elif pos in ['DEF', 'MID']:
                model += pulp.lpSum(lineup[p, gw] for p in players if player_data.get((p, gw), {}).get('position') == pos) >= 3
                model += pulp.lpSum(lineup[p, gw] for p in players if player_data.get((p, gw), {}).get('position') == pos) <= 5
            else:  # FWD
                model += pulp.lpSum(lineup[p, gw] for p in players if player_data.get((p, gw), {}).get('position') == pos) >= 1
                model += pulp.lpSum(lineup[p, gw] for p in players if player_data.get((p, gw), {}).get('position') == pos) <= 3

        # Budget constraint
        model += pulp.lpSum(player_data.get((p, gw), {}).get('value', 0) * squad[p, gw] for p in players) <= budget

        # Team limit
        for team in teams:
            model += pulp.lpSum(squad[p, gw] for p in players if player_data.get((p, gw), {}).get('team') == team) <= 3

        # Transfer constraints
        if gw > start_gameweek:
            model += free_transfers[gw] == free_transfers[gw-1] + 1 - pulp.lpSum(transfer_in[p, gw-1] for p in players)
            model += free_transfers[gw] <= 5 
            model += pulp.lpSum(transfer_in[p, gw] for p in players) == pulp.lpSum(transfer_out[p, gw] for p in players)
            model += pulp.lpSum(transfer_in[p, gw] for p in players) == free_transfers[gw] + paid_transfers[gw]

            
        for p in players:
            if gw > start_gameweek:
                model += squad[p, gw] == squad[p, gw-1] + transfer_in[p, gw] - transfer_out[p, gw]
            model += transfer_in[p, gw] + transfer_out[p, gw] <= 1

    # Solve the model
    model.solve()

    # Process results
    if pulp.LpStatus[model.status] == 'Optimal':
        results = []
        for gw in gameweeks:
            picks = []
            transfers_in = []
            transfers_out = []
            for p in players:
                if squad[p, gw].value() > 0.5:
                    player_info = player_data.get((p, gw), {})
                    is_captain = 1 if captain[p, gw].value() > 0.5 else 0
                    is_lineup = 1 if lineup[p, gw].value() > 0.5 else 0
                    is_vice = 1 if vicecap[p, gw].value() > 0.5 else 0
                    picks.append([
                        p,
                        player_info.get('position', ''),
                        player_info.get('team', ''),
                        player_info.get('value', 0),
                        player_info.get('xP', 0),
                        is_lineup,
                        is_captain,
                        is_vice
                    ])
                if gw > start_gameweek:
                    if transfer_in[p, gw].value() > 0.5:
                        transfers_in.append(p)
                    if transfer_out[p, gw].value() > 0.5:
                        transfers_out.append(p)

            picks_df = pd.DataFrame(picks, columns=['name', 'pos', 'team', 'price', 'xP', 'lineup', 'captain', 'vicecaptain'])
            picks_df = picks_df.sort_values(by=['lineup', 'pos', 'xP'], ascending=[False, True, False])
            
            results.append({
                'gameweek': gw,
                'picks': picks_df,
                'transfers_in': transfers_in,
                'transfers_out': transfers_out,
                'free_transfers': free_transfers[gw].value(),
                'paid_transfers': paid_transfers[gw].value()
            })

        total_xp = pulp.value(model.objective)
        print(f'Total expected points for all gameweeks: {total_xp}')

        return {'results': results, 'total_xp': total_xp}
    else:
        print(f"Optimization failed. Status: {pulp.LpStatus[model.status]}")
        return None

In [19]:
result = solve_multi_period_fpl(100, df_optimisation, 1, 6, '2024-25')

if result:
    for gw_result in result['results']:
        print(f"Gameweek {gw_result['gameweek']}:")
        print(gw_result['picks'])
        if gw_result['gameweek'] > 1:
            print("Transfers In:", gw_result['transfers_in'])
            print("Transfers Out:", gw_result['transfers_out'])
        print(f"Free Transfers: {gw_result['free_transfers']}")
        print(f"Paid Transfers: {gw_result['paid_transfers']}")
        print("\n")

    print(f"Total Expected Points: {result['total_xp']}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/85881e3b25a544e599570a3a0d36322a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/85881e3b25a544e599570a3a0d36322a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 19772 COLUMNS
At line 154769 RHS
At line 174537 BOUNDS
At line 194638 ENDATA
Problem MODEL has 19767 rows, 20100 columns and 85610 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 376.332 - 0.19 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 10044 strengthened rows, 0 substitutions
Cgl0004I processed model has 12526 rows, 18982 columns (18982 integer (18973 of which binary)) and 70998 elements
Cutoff increment in

In [20]:
import pandas as pd

# Assuming 'result' contains the output from solve_multi_period_fpl
if result:
    markdown_output = ""

    # Add picks for each gameweek to Markdown
    for gw_result in result['results']:
        markdown_output += f"### Gameweek {gw_result['gameweek']} Picks\n\n"
        markdown_output += gw_result['picks'].to_markdown(index=False)
        markdown_output += "\n\n"

    # Add transfers summary to Markdown
    markdown_output += "### Transfers Summary\n\n"

    transfer_records = []
    for gw_result in result['results'][1:]:
        transfers_in = gw_result['transfers_in']
        transfers_out = gw_result['transfers_out']
        max_transfers = max(len(transfers_in), len(transfers_out))

        for i in range(max_transfers):
            transfer_in = transfers_in[i] if i < len(transfers_in) else ''
            transfer_out = transfers_out[i] if i < len(transfers_out) else ''
            transfer_records.append({'Gameweek': gw_result['gameweek'], 'Transfers In': transfer_in, 'Transfers Out': transfer_out})

    # Create a single DataFrame for all transfers
    transfers_df = pd.DataFrame(transfer_records)

    # Convert transfers DataFrame to Markdown
    markdown_output += transfers_df.to_markdown(index=False)

    # Print the Markdown output
    print(markdown_output)
else:
    print("Optimization did not find an optimal solution.")

### Gameweek 1 Picks

| name             | pos   | team   |   price |   xP |   lineup |   captain |   vicecaptain |
|:-----------------|:------|:-------|--------:|-----:|---------:|----------:|--------------:|
| Trippier         | DEF   | NEW    |     6   |  5.2 |        1 |         0 |             0 |
| Alexander-Arnold | DEF   | LIV    |     7   |  4.8 |        1 |         0 |             0 |
| Pedro Porro      | DEF   | TOT    |     5.5 |  4.2 |        1 |         0 |             0 |
| Mateta           | FWD   | CRY    |     7.5 |  4.4 |        1 |         0 |             0 |
| Muniz            | FWD   | FUL    |     6   |  3.8 |        1 |         0 |             0 |
| Muric            | GK    | IPS    |     4.5 |  3.5 |        1 |         0 |             0 |
| M.Salah          | MID   | LIV    |    12.5 |  7   |        1 |         1 |             0 |
| Saka             | MID   | ARS    |    10   |  6.8 |        1 |         0 |             1 |
| Palmer           | MID   | CHE    | 